# LP Code

## 常见的解决LP问题的Python工具包

- Scipy
- PuLP

## 工具包介绍

SciPy是一个开源的Python算法库和数学工具包。

SciPy包含的模块有最优化、线性代数、积分、插值、特殊函数、快速傅里叶变换、信号处理和图像处理、常微分方程求解和其他科学与工程中常用的计算。

PuLP是一个开源的线性规划包


## 使用方法


### scipy
![title](img/5.png)


其中，

- c为线性目标函数的系数
- A_ub和b_ub对应线性不等式约束
- A_eq和b_eq对应线性等式约束
- bounds确定边界，如x≥0为（0，None）x无约束则为（None，None）
- method是求解器的类型，'revised simplex’为修改的单纯形法

要使用linprog,目标函数要变成求最小值，如果原题目要求求max（最大值），只需对目标函数取负，但要注意求解的最终值是取负后的目标函数的最小值，取负即为最大值

![title](img/6.png)

![title](img/7.png)

### PuLP

- 导入库函数

```
from pulp import *
```

- 定义线性规划问题

```
PB = LpProblem(problem name, sense)
```

构造一个LP问题实例，其实name指定问题名，sense指是LpMaximize, LpMinimize 中的一个，用来指定目标函数是求最大值还是最小值

- 定义决策变量

```
DV = LpVariable(decision variable name, lowbound, upbound, category)

```

decision variable name 用来指定变量名，lowbound 和 upbound是下界和上界，默认分别是负无穷和正无穷，category用来指定变量是离散还是连续

- 添加目标函数

```
PB += linear object from object name
```

- 添加约束条件

```
PB += linear object from constraint name 
```

- 简单举例

```
x = LpVariable("x", 0, 3)
y = LpVariable("y", 0, 1)
prob = LpProblem("myProblem", LpMinimize)
prob += x + y <= 2
prob += -4*x + y
```

- 写入LP文件

```
PB.writeLP(filename)
```

- 模型求解

```
PB.solver()
```

可以是用不同的solver

- 结果显示

```
LpStatus(PB.status)
```



## 采购问题

原题目：
有2000元经费，需要采购单价为50元的若干桌子和单价为20元的若干椅子，你希望桌椅的总数尽可能的多，

但要求椅子数量不少于桌子数量，且不多于桌子数量的1.5倍，那你需要怎样的一个采购方案呢？

解：要采购x1张桌子，x2把椅子
```
max z= x1 + x2
s.t. x1 - x2 <= 0
1.5x1 >= x2
50x1 + 20x2 <= 2000
x1, x2 >=0
```

求解第一步，转换成标准形式

```
min  -x1 - x2
s.t. x1 - x2 <= 0
-1.5x1 + x2 <= 0
50x1 + 20x2 <= 2000
x1, x2 >=0
```

### Scipy solver

```
在python中此类线性规划问题可用lp solver解决
scipy.optimize._linprog def linprog(c: int,
            A_ub: Optional[int] = None,
            b_ub: Optional[int] = None,
            A_eq: Optional[int] = None,
            b_eq: Optional[int] = None,
            bounds: Optional[Iterable] = None,
            method: Optional[str] = 'simplex',
            callback: Optional[Callable] = None,
            options: Optional[dict] = None) -> OptimizeResult

矩阵A：就是约束条件的系数（等号左边的系数）
矩阵B：就是约束条件的值（等号右边）
矩阵C：目标函数的系数值
```

In [2]:
from scipy import  optimize as opt
import numpy as np
#参数
#c是目标函数里变量的系数
c=np.array([1,1])
#a是不等式条件的变量系数
a=np.array([[1,-1],[-1.5,1],[50,20]])
#b是是不等式条件的常数项
b=np.array([0,0,2000])
#a1，b1是等式条件的变量系数和常数项，这个例子里无等式条件,不要这两项
#a1=np.array([[1,1,1]])
#b1=np.array([7])
#限制
lim1=(0,None) #(0,None)->(0,+无穷)
lim2=(0,None)
#调用函数
ans=opt.linprog(-c,a,b,bounds=(lim1,lim2),method='revised simplex')
#输出结果
print(ans)

#注意：我们这里的应用问题，椅子不能是0.5把，所以最后应该采购37把椅子


     con: array([], dtype=float64)
     fun: -62.5
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([12.5,  0. ,  0. ])
  status: 0
 success: True
       x: array([25. , 37.5])


### Pulp

In [4]:
from pulp import *
PB = LpProblem("Shopping Problem", LpMinimize)
x1 = LpVariable("x1", lowBound=0, cat=LpInteger)
x2 = LpVariable("x2", lowBound=0, cat=LpInteger)
PB += x1 - x2 <= 0
PB += -1.5 * x1 + x2 <= 0
PB += 50 * x1 + 20* x2 <= 2000
PB += -x1 - x2

/Users/supinyu/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
PB

Shopping_Problem:
MINIMIZE
-1*x1 + -1*x2 + 0
SUBJECT TO
_C1: x1 - x2 <= 0

_C2: - 1.5 x1 + x2 <= 0

_C3: 50 x1 + 20 x2 <= 2000

VARIABLES
0 <= x1 Integer
0 <= x2 Integer

In [12]:
status = PB.solve()

In [13]:
LpStatus[status]

'Optimal'

In [14]:
value(x1)

25.0

In [15]:
value(x2)

37.0

In [16]:
for variable in PB.variables():
    print("{} = {}".format(variable.name, variable.varValue))

print(value(PB.objective))

x1 = 25.0
x2 = 37.0
-62.0


## 最大流问题

![title](img/8.png)

```
如图，假设有四个城市，s，u，v，t。
连线表示，城市之间有道路。连线上的数字表示这条道路每天最多能够运送货物的吨数。

比如，城市 s，u 之间有条路，这条路每天最多运 5 吨货物。
如果你是调度员，请设计一个调度方案，
使得从 s 到 t 一天之内能够运送的货物越多越好。

现在我们形式化这个问题，将 5 条路的运输量分别为 x1，x2，x3，x4，x5。
然后写出约束条件和目标函数
```

```
max x1 + x2
s.t. x1 - x3 - x4 = 0  node U
x2 + x3 - x5 = 0  node V
5 ≥ x1 ≥ 0
8 ≥ x2 ≥ 0
1 ≥ x3 ≥ 0
6 ≥ x4 ≥ 0
2 ≥ x5 ≥ 0
```
因为 u，v 是中间节点，所以必须满足运入 u，v 的货物必须在当天运
出。

另外要满足每条路的运输量不超过其最大运量。我们的目标是要从 s 运出的货
物越多越好。

### scipy

In [24]:
from scipy import  optimize as opt
import numpy as np
#参数
#c是目标函数里变量的系数
c=np.array([1,1,0,0,0])
#a是不等式条件的变量系数
a=np.array([[1,0,-1,-1,0],[0,1,1,0,-1]])
#b是是不等式条件的常数项
b=np.array([0,0])
#限制
lim1 = (0,5) #(0,None)->(0,+无穷)
lim2 = (0,8)
lim3 = (0,1)
lim4 = (0,6)
lim5 = (0,2)
#调用函数
ans=opt.linprog(c = -c, A_eq = a, b_eq=b, bounds = [lim1,lim2,lim3,lim4,lim5], method='revised simplex')
#输出结果
print(ans)

     con: array([0., 0.])
     fun: -7.0
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([5., 2., 0., 5., 2.])


### Pulp

In [26]:
from pulp import *
PB = LpProblem("Max Flow Problem", LpMinimize)
x1 = LpVariable("x1", lowBound=0, upBound=5, cat=LpContinuous)
x2 = LpVariable("x2", lowBound=0, upBound=8, cat=LpContinuous)
x3 = LpVariable("x3", lowBound=0, upBound=1, cat=LpContinuous)
x4 = LpVariable("x4", lowBound=0, upBound=6, cat=LpContinuous)
x5 = LpVariable("x5", lowBound=0, upBound=2, cat=LpContinuous)
PB += x1 - x3 - x4 <= 0
PB += -x1 + x3 + x4 <= 0
PB += x2 + x3 - x5 <= 0
PB += -x2 - x3 + x5 <= 0
PB += -x1 - x2

/Users/supinyu/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [27]:
PB

Max_Flow_Problem:
MINIMIZE
-1*x1 + -1*x2 + 0
SUBJECT TO
_C1: x1 - x3 - x4 <= 0

_C2: - x1 + x3 + x4 <= 0

_C3: x2 + x3 - x5 <= 0

_C4: - x2 - x3 + x5 <= 0

VARIABLES
x1 <= 5 Continuous
x2 <= 8 Continuous
x3 <= 1 Continuous
x4 <= 6 Continuous
x5 <= 2 Continuous

In [28]:
status = PB.solve()

In [29]:
LpStatus[status]

'Optimal'

In [30]:
for variable in PB.variables():
    print("{} = {}".format(variable.name, variable.varValue))

print(value(PB.objective))

x1 = 5.0
x2 = 2.0
x3 = 0.0
x4 = 5.0
x5 = 2.0
-7.0


### 参考资料

- https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

- https://www.hrwhisper.me/introduction-to-simplex-algorithm/